In [1]:
import pandas as pd
import math
import os

In [2]:
data_url = "./data/단향 측정 데이터_권혁진.xlsx"
df_dict = pd.read_excel(data_url, sheet_name=None)
print(df_dict)

{'Heptanol':       Timesta+A1:J1mps (sec)       ZnO (1)       ZnO (2)       ZnO (3)  \
0                   0.000009  7.740000e-10  3.028200e-07  1.332600e-08   
1                   2.981646  1.150800e-08  2.955960e-07  1.503100e-08   
2                   5.951279  1.164000e-08  2.985380e-07  1.300000e-08   
3                   8.889494  1.139700e-08  3.051830e-07  1.305800e-08   
4                  11.895487  8.972000e-09  2.375300e-07  7.302000e-09   
...                      ...           ...           ...           ...   
6868            20811.378324  1.416000e-08  2.425700e-07  1.559200e-08   
6869            20814.387244  1.684000e-08  2.796200e-07  1.222800e-08   
6870            20817.388022  1.315500e-08  2.147550e-07  2.374100e-08   
6871            20820.404342  1.562500e-08  2.394680e-07  7.314000e-09   
6872            20823.416638  1.898300e-08           NaN           NaN   

      SnO2 (1)  SnO2 (2)  SnO2 (3)       AZO (1)       AZO (2)       AZO (3)  
0     0.000072  0.0

In [3]:
first_scent_name = list(df_dict.keys())[0]
# Where we are going to integrate scent data in integrated data
# Make a "scent_name" column.
integrated_data = pd.DataFrame(columns= df_dict[first_scent_name].columns)
integrated_data["scent_name"] = None

# concat the df of each scent into integrated data.
for scent_name in df_dict.keys():
    df_dict[scent_name]["scent_name"] = scent_name
    integrated_data = pd.concat([integrated_data, df_dict[scent_name]], ignore_index = True)  

print(integrated_data.head())
print(integrated_data.shape)
print(integrated_data["scent_name"].unique())


   Timesta+A1:J1mps (sec)       ZnO (1)       ZnO (2)       ZnO (3)  SnO2 (1)  \
0                0.000009  7.740000e-10  3.028200e-07  1.332600e-08  0.000072   
1                2.981646  1.150800e-08  2.955960e-07  1.503100e-08  0.000072   
2                5.951279  1.164000e-08  2.985380e-07  1.300000e-08  0.000073   
3                8.889494  1.139700e-08  3.051830e-07  1.305800e-08  0.000073   
4               11.895487  8.972000e-09  2.375300e-07  7.302000e-09  0.000073   

   SnO2 (2)  SnO2 (3)       AZO (1)   AZO (2)       AZO (3) scent_name  
0  0.000065  0.000053  1.267340e-07  0.000001  4.576990e-07   Heptanol  
1  0.000064  0.000054  1.277770e-07  0.000001  5.968640e-07   Heptanol  
2  0.000065  0.000054  1.306230e-07  0.000001  5.696490e-07   Heptanol  
3  0.000065  0.000054  1.175820e-07  0.000001  5.729060e-07   Heptanol  
4  0.000066  0.000055  1.184220e-07  0.000001  5.641550e-07   Heptanol  
(36670, 11)
['Heptanol' '2-ethylfenchol' 'Octanal' 'Eugenol' 'Cis-3-hexenol

In [4]:
# Change Column name to "Time (sec)"
integrated_data = integrated_data.rename(columns = {"Timesta+A1:J1mps (sec)" :"Time (sec)"})
integrated_data.head()

,Time (sec),ZnO (1),ZnO (2),ZnO (3),SnO2 (1),SnO2 (2),SnO2 (3),AZO (1),AZO (2),AZO (3),scent_name
0,0.000009,7.740000e-10,3.028200e-07,1.332600e-08,0.000072,0.000065,0.000053,1.267340e-07,0.000001,4.576990e-07,Heptanol
1,2.981646,1.150800e-08,2.955960e-07,1.503100e-08,0.000072,0.000064,0.000054,1.277770e-07,0.000001,5.968640e-07,Heptanol
2,5.951279,1.164000e-08,2.985380e-07,1.300000e-08,0.000073,0.000065,0.000054,1.306230e-07,0.000001,5.696490e-07,Heptanol
3,8.889494,1.139700e-08,3.051830e-07,1.305800e-08,0.000073,0.000065,0.000054,1.175820e-07,0.000001,5.729060e-07,Heptanol
4,11.895487,8.972000e-09,2.375300e-07,7.302000e-09,0.000073,0.000066,0.000055,1.184220e-07,0.000001,5.641550e-07,Heptanol


In [5]:
# bubble is repeated by off/on.
# use math.inf to make a boundary for off in this case (+ 600, 1200)
nose_open_close_pattern = [(0, 600), (1200, 2400), (3000, 4200), (4800, 6000), (6600, 7800), (8400, 9600), (10200, 11400), (12000, 13200), (13800, 15000), (15600, 16800), (17400, 18600), (19200, math.inf)]
# Heptanol is in A --> we are going to use nose_...A

#make a defalut column filled with "on"
integrated_data["on_off_state"] = "on"

for i, j in nose_open_close_pattern:
    integrated_data["on_off_state"] = integrated_data.apply(lambda x: "off" if (i <= x["Time (sec)"]) & (x["Time (sec)"] < j) else x["on_off_state"], axis=1)

integrated_data.head()

,Time (sec),ZnO (1),ZnO (2),ZnO (3),SnO2 (1),SnO2 (2),SnO2 (3),AZO (1),AZO (2),AZO (3),scent_name,on_off_state
0,0.000009,7.740000e-10,3.028200e-07,1.332600e-08,0.000072,0.000065,0.000053,1.267340e-07,0.000001,4.576990e-07,Heptanol,off
1,2.981646,1.150800e-08,2.955960e-07,1.503100e-08,0.000072,0.000064,0.000054,1.277770e-07,0.000001,5.968640e-07,Heptanol,off
2,5.951279,1.164000e-08,2.985380e-07,1.300000e-08,0.000073,0.000065,0.000054,1.306230e-07,0.000001,5.696490e-07,Heptanol,off
3,8.889494,1.139700e-08,3.051830e-07,1.305800e-08,0.000073,0.000065,0.000054,1.175820e-07,0.000001,5.729060e-07,Heptanol,off
4,11.895487,8.972000e-09,2.375300e-07,7.302000e-09,0.000073,0.000066,0.000055,1.184220e-07,0.000001,5.641550e-07,Heptanol,off


In [6]:
# GOALS : Make a on_off_state and Trial_numbers
# bubble is repeated by off/on.
# use math.inf to make a boundary for off in this case (+ 600, 1200)
nose_open_close_pattern = [0, 600, 1200, 2400, 3000, 4200, 4800, 6000, 6600, 7800, 8400, 9600, 10200, 11400, 12000, 13200, 13800, 15000, 15600, 16800, 17400, 18600, 19200, math.inf]
# 1. "on_off_state"
#make a defalut column filled with "on"
integrated_data["on_off_state"] = "on"

for i in range(len(nose_open_close_pattern)//2):
    integrated_data["on_off_state"] = integrated_data.apply(lambda x: "off" if (nose_open_close_pattern[2*i] <= x["Time (sec)"]) & (x["Time (sec)"] < nose_open_close_pattern[2*i + 1]) else x["on_off_state"], axis=1)

# 2. "trial_num"
# set trial_num as 0 series.
integrated_data["trial_num"] = 0
for i in range(len(nose_open_close_pattern)//2 - 1):
    integrated_data["trial_num"] = integrated_data.apply(lambda x: i + 1 if (nose_open_close_pattern[2*i] <= x["Time (sec)"]) & (x["Time (sec)"] < nose_open_close_pattern[2*i + 2]) else x["trial_num"] ,axis= 1)

integrated_data.head()

,Time (sec),ZnO (1),ZnO (2),ZnO (3),SnO2 (1),SnO2 (2),SnO2 (3),AZO (1),AZO (2),AZO (3),scent_name,on_off_state,trial_num
0,0.000009,7.740000e-10,3.028200e-07,1.332600e-08,0.000072,0.000065,0.000053,1.267340e-07,0.000001,4.576990e-07,Heptanol,off,1
1,2.981646,1.150800e-08,2.955960e-07,1.503100e-08,0.000072,0.000064,0.000054,1.277770e-07,0.000001,5.968640e-07,Heptanol,off,1
2,5.951279,1.164000e-08,2.985380e-07,1.300000e-08,0.000073,0.000065,0.000054,1.306230e-07,0.000001,5.696490e-07,Heptanol,off,1
3,8.889494,1.139700e-08,3.051830e-07,1.305800e-08,0.000073,0.000065,0.000054,1.175820e-07,0.000001,5.729060e-07,Heptanol,off,1
4,11.895487,8.972000e-09,2.375300e-07,7.302000e-09,0.000073,0.000066,0.000055,1.184220e-07,0.000001,5.641550e-07,Heptanol,off,1


## Let's make columns more efficient to access.  
Hieracrchical columns like ZnO > 1, 2, 3 

In [65]:
integrated_data_optimized = integrated_data.copy()

In [66]:
# integrated_data.set_axis(["Time (sec)", [["Zno"], [1]], [["Zno"], [2]], [["Zno"], [3]], [["SnO2"], [1]], [["SnO2"], [2]], [["SnO2"], [3]], [["AZO"], [1]], [["AZO"], [2]], [["AZO"], [3]], "scent_name", "on_off_state", "trial_name" ] ,axis="columns")

# integrated_data_optimized.iloc[0] = [None, "1", "2", "3", "1", "2", "3", "1", "2", "3", None, None, None]

integrated_data_optimized = integrated_data_optimized.set_index(["scent_name", "trial_num", "on_off_state", "Time (sec)"])


integrated_data_optimized.head()

ZnO (1)       ZnO (2)  \
scent_name trial_num on_off_state Time (sec)                               
Heptanol   1         off          0.000009    7.740000e-10  3.028200e-07   
                                  2.981646    1.150800e-08  2.955960e-07   
                                  5.951279    1.164000e-08  2.985380e-07   
                                  8.889494    1.139700e-08  3.051830e-07   
                                  11.895487   8.972000e-09  2.375300e-07   

                                                   ZnO (3)  SnO2 (1)  \
scent_name trial_num on_off_state Time (sec)                           
Heptanol   1         off          0.000009    1.332600e-08  0.000072   
                                  2.981646    1.503100e-08  0.000072   
                                  5.951279    1.300000e-08  0.000073   
                                  8.889494    1.305800e-08  0.000073   
                                  11.895487   7.302000e-09  0.000073   

                                              SnO2 (2)  SnO2 (3)  \
scent_name trial_num on_off_state Time (sec)                       
Heptanol   1         off          0.000009    0.000065  0.000053   
                                  2.981646    0.000064  0.000054   
                                  5.951279    0.000065  0.000054   
                                  8.889494    0.000065  0.000054   
                                  11.895487   0.000066  0.000055   

                                                   AZO (1)   AZO (2)  \
scent_name trial_num on_off_state Time (sec)                           
Heptanol   1         off          0.000009    1.267340e-07  0.000001   
                                  2.981646    1.277770e-07  0.000001   
                                  5.951279    1.306230e-07  0.000001   
                                  8.889494    1.175820e-07  0.000001   
                                  11.895487   1.184220e-07  0.000001   

                                                   AZO (3)  
scent_name trial_num on_off_state Time (sec)                
Heptanol   1         off          0.000009    4.576990e-07  
                                  2.981646    5.968640e-07  
                                  5.951279    5.696490e-07  
                                  8.889494    5.729060e-07  
                                  11.895487   5.641550e-07

In [67]:
integrated_data_optimized = integrated_data_optimized.set_axis([integrated_data_optimized.columns.map(lambda x: x[:-4]), pd.Index([1, 2, 3]*3) ], axis = 1)

integrated_data_optimized.head()
# integrated_data_optimized.columns = [integrated_data_optimized.columns[0]] + list(integrated_data_optimized.columns[1:10].map(lambda x: x[:3])) + list(integrated_data_optimized.columns[10:], list(integrated_data_optimized.iloc[0]))

ZnO                \
                                                         1             2   
scent_name trial_num on_off_state Time (sec)                               
Heptanol   1         off          0.000009    7.740000e-10  3.028200e-07   
                                  2.981646    1.150800e-08  2.955960e-07   
                                  5.951279    1.164000e-08  2.985380e-07   
                                  8.889494    1.139700e-08  3.051830e-07   
                                  11.895487   8.972000e-09  2.375300e-07   

                                                                SnO2  \
                                                         3         1   
scent_name trial_num on_off_state Time (sec)                           
Heptanol   1         off          0.000009    1.332600e-08  0.000072   
                                  2.981646    1.503100e-08  0.000072   
                                  5.951279    1.300000e-08  0.000073   
                                  8.889494    1.305800e-08  0.000073   
                                  11.895487   7.302000e-09  0.000073   

                                                                  \
                                                     2         3   
scent_name trial_num on_off_state Time (sec)                       
Heptanol   1         off          0.000009    0.000065  0.000053   
                                  2.981646    0.000064  0.000054   
                                  5.951279    0.000065  0.000054   
                                  8.889494    0.000065  0.000054   
                                  11.895487   0.000066  0.000055   

                                                       AZO            \
                                                         1         2   
scent_name trial_num on_off_state Time (sec)                           
Heptanol   1         off          0.000009    1.267340e-07  0.000001   
                                  2.981646    1.277770e-07  0.000001   
                                  5.951279    1.306230e-07  0.000001   
                                  8.889494    1.175820e-07  0.000001   
                                  11.895487   1.184220e-07  0.000001   

                                                            
                                                         3  
scent_name trial_num on_off_state Time (sec)                
Heptanol   1         off          0.000009    4.576990e-07  
                                  2.981646    5.968640e-07  
                                  5.951279    5.696490e-07  
                                  8.889494    5.729060e-07  
                                  11.895487   5.641550e-07

In [68]:
integrated_data_optimized.index


MultiIndex([('Heptanol', 1, 'off',       9e-06),
            ('Heptanol', 1, 'off',    2.981646),
            ('Heptanol', 1, 'off',    5.951279),
            ('Heptanol', 1, 'off',    8.889494),
            ('Heptanol', 1, 'off',   11.895487),
            ('Heptanol', 1, 'off',   14.882415),
            ('Heptanol', 1, 'off',   17.813674),
            ('Heptanol', 1, 'off',   20.771962),
            ('Heptanol', 1, 'off',   23.768765),
            ('Heptanol', 1, 'off',   26.772708),
            ...
            ('Geraniol', 6,  'on', 9859.097593),
            ('Geraniol', 6,  'on', 9862.108969),
            ('Geraniol', 6,  'on', 9865.114172),
            ('Geraniol', 6,  'on', 9868.139747),
            ('Geraniol', 6,  'on', 9871.127468),
            ('Geraniol', 6,  'on', 9874.223003),
            ('Geraniol', 6,  'on', 9877.246378),
            ('Geraniol', 6,  'on', 9880.249852),
            ('Geraniol', 6,  'on',  9883.25769),
            ('Geraniol', 6,  'on', 9886.271773)],
   

In [72]:
integrated_data_optimized.loc["Heptanol", 1, "on"]

ipykernel_launcher:1: PerformanceWarning: indexing past lexsort depth may impact performance.


ZnO                                  SnO2            \
                        1             2             3         1         2   
Time (sec)                                                                  
602.290883   1.184100e-08  4.729630e-07  2.050200e-08  0.000075  0.000071   
605.298575   1.066900e-08  4.498490e-07  1.905700e-08  0.000075  0.000071   
608.322897   1.154800e-08  4.881370e-07  1.891100e-08  0.000075  0.000071   
611.334691   1.160100e-08  4.059930e-07  1.919600e-08  0.000075  0.000072   
614.342759   1.175700e-08  4.858830e-07  1.987100e-08  0.000075  0.000071   
...                   ...           ...           ...       ...       ...   
1187.330076  4.290000e-09  6.416600e-07  2.171500e-08  0.000076  0.000042   
1190.345655  4.245000e-09  5.968000e-07  2.134100e-08  0.000076  0.000042   
1193.340548  4.270000e-09  6.659500e-07  2.116500e-08  0.000076  0.000041   
1196.346888 -1.652000e-09  5.601830e-07  1.873300e-08  0.000076  0.000042   
1199.346527  4.799000e-09  5.743000e-07  2.099000e-08  0.000076  0.000042   

                                AZO                          
                    3             1         2             3  
Time (sec)                                                   
602.290883   0.000058  1.418540e-07  0.000001  5.768280e-07  
605.298575   0.000058  1.403730e-07  0.000001  5.713140e-07  
608.322897   0.000058  1.375820e-07  0.000001  5.731830e-07  
611.334691   0.000059  1.414050e-07  0.000001  5.600070e-07  
614.342759   0.000058  1.372530e-07  0.000001  5.713950e-07  
...               ...           ...       ...           ...  
1187.330076  0.000055  1.934740e-07  0.000002  1.083730e-06  
1190.345655  0.000055  1.929970e-07  0.000002  1.064790e-06  
1193.340548  0.000055  1.856380e-07  0.000002  1.044980e-06  
1196.346888  0.000055  1.914900e-07  0.000002  1.045810e-06  
1199.346527  0.000055  1.957650e-07  0.000002  1.042310e-06  

[199 rows x 9 columns]

In [73]:
integrated_data["trial_num"].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,  0], dtype=int64)

0 = Not in the trials. 

## Trials made in each scent

In [74]:
for name in integrated_data["scent_name"].unique():
    trials = integrated_data[integrated_data["scent_name"] == name]["trial_num"].max()
    print(f"trials made in {name}: {trials}")

trials made in Heptanol: 11
trials made in 2-ethylfenchol: 11
trials made in Octanal: 11
trials made in Eugenol: 6
trials made in Cis-3-hexenol: 6
trials made in Decanal: 6
trials made in 2,3,5 trymethylpyrazine: 8
trials made in Geraniol: 6


In [77]:
# save as .csv
path = "./data/preprocessed_data.csv"
integrated_data.to_csv(path, index= False)

In [80]:
# pickle can save the multi index
integrated_data_optimized.to_pickle("./data/preprocessed_data.pkl")